In [ ]:
import propagators
import energy_landscapes
import long_simulation
import weighted_ensemble
import numpy as np

kT = 1
x_init_coord = -1/np.sqrt(2)
dt = 0.001
nsteps = 10000
save_period = 10 #in steps
n_parallel = 10
nbins = 40
system1 = energy_landscapes.unit_double_well()


In [ ]:
#system1.plot_quantity(system1.potential)
xd, ed, rp = long_simulation.recover_energy_landscape(propagators.propagate_nd, system1, kT, x_init_coord, dt, nsteps, save_period, n_parallel, nbins)
print(f"simulation steps:\n Aggregate: {nsteps*n_parallel} \n Molecular: {nsteps}")


In [ ]:
#Weighted ensemble (WE) parameters and inputs

N = 80             #total number of walkers within binrange
nbins = 40         #total number of bins within binrange
binrange = [-1.5,1.5] #progress coordinate range within which to bin simulations
                    #this should extend well past the stall point for examination of the WE stall force
                    #the area past either end of binrange is a bin extending to either + or - inf, yielding a total of nbins+2 bins
n_macrostates=2
        
nsteps = 200        #round length
nrounds = 50        #number of WE rounds to run

walkers_per_bin = round(N/nbins)
print(f"Each bin can hold up to {walkers_per_bin} walkers, for a total of up to {walkers_per_bin*(nbins+2)} walkers")

#start 1 bin worth of walkers at x=0 with equal weights
x_init = np.array([-1/np.sqrt(2) for element in range(walkers_per_bin)])
w_init = [1/walkers_per_bin for element in range(walkers_per_bin)]

#run weighted ensemble with brownian dynamics
#put this on multiple lines
x_init, e_init, w_init, binbounds, xtrj, etrj, wtrj, transitions, hamsm_transitions, n_trans_by_round \
= weighted_ensemble.weighted_ensemble(\
                    x_init,\
                    w_init,\
                    nrounds,\
                    nbins,\
                    walkers_per_bin,\
                    binrange, propagators.propagate_nd_save1,\
                    [system1, kT, dt, nsteps],\
                    system1.macro_class,\
                    n_macrostates,\
                    ha_binning=False)


weighted_ensemble.landscape_recovery(xtrj, wtrj, binbounds, transitions, hamsm_transitions, n_trans_by_round, nrounds, n_macrostates, system1.potential, system1.macro_class, kT)

aggregate_walkers = len([j for i in xtrj for j in i])
print(f"simulation steps:\n Aggregate: {nsteps*aggregate_walkers} \n Molecular: {nsteps*nrounds}")


In [ ]:
w_trj_flat = [j for i in wtrj for j in i]
import matplotlib.pyplot as plt
plt.hist(w_trj_flat)
#more aggressive methods are needed to keep walker weights in a reasonable range

In [ ]:
a = sorted(w_trj_flat)
print(a)
a.reverse()
print(a)

In [ ]:
plt.plot([sum(a[0:i]) for i in range(len(a))])

In [ ]:
a = [1,.5,3]
np.argsort(a)